# Stroke Prediction Project
In this project, the aim is have a good result with the parameter put in this model
from the result of sensor

In [81]:
#Install libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("./healthcare-dataset-stroke-data.csv", index_col = "id")

In [3]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
#Data wragling
data.info()
#Result the bmi column has NaN values.

<class 'pandas.core.frame.DataFrame'>
Index: 5110 entries, 9046 to 44679
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 479.1+ KB


In [7]:
# Spliting the dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
le = LabelEncoder()
transform_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

for col in transform_columns:
    data[col] = le.fit_transform(data[col])
data

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
51676,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
31112,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
60182,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
1665,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
18234,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
44873,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
19723,0,35.0,0,0,1,3,0,82.99,30.600000,2,0


In [8]:
data['stroke'].value_counts()

stroke
0    4861
1     249
Name: count, dtype: int64

In [39]:
# -> divide the dataset into percentage 7-3 to balance the test
from sklearn.utils import resample
data_normal = data[data['stroke'] == 0]
data_obnormal = data[data['stroke'] == 1]

data_normal_down = resample(data_normal, 
                           replace = False, 
                           n_samples = (int)(len(data_obnormal) * 1.5),
                           random_state = 42)
data_sum = pd.concat([data_normal_down, data_obnormal])
# Mix the data to ensure random value will be used ( 0 and 1 )
data_sum = data_sum.sample(frac=1, random_state=42).reset_index(drop=True)
data_sum

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,40.0,0,0,1,0,1,114.32,28.3,3,0
1,0,65.0,0,0,1,2,0,100.98,28.2,1,1
2,0,29.0,0,0,1,2,0,102.07,31.8,2,0
3,0,68.0,1,0,1,2,1,95.82,28.6,2,0
4,0,54.0,0,0,1,0,1,74.23,28.1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
617,0,56.0,0,0,1,2,1,66.32,23.4,2,0
618,1,67.0,0,0,1,2,0,65.51,33.2,1,0
619,0,8.0,0,0,0,4,1,87.15,16.1,0,0
620,0,67.0,1,0,1,3,0,61.94,25.3,3,1


In [40]:
#choose features for x and y
x = data_sum.drop(columns = 'stroke')
y = data_sum['stroke']
print(x, y)
print(y.value_counts())

     gender   age  hypertension  heart_disease  ever_married  work_type  \
0         0  40.0             0              0             1          0   
1         0  65.0             0              0             1          2   
2         0  29.0             0              0             1          2   
3         0  68.0             1              0             1          2   
4         0  54.0             0              0             1          0   
..      ...   ...           ...            ...           ...        ...   
617       0  56.0             0              0             1          2   
618       1  67.0             0              0             1          2   
619       0   8.0             0              0             0          4   
620       0  67.0             1              0             1          3   
621       1  13.0             0              0             0          4   

     Residence_type  avg_glucose_level   bmi  smoking_status  
0                 1             114.

In [41]:
#Spliting the dataset

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [42]:
x_train

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
209,1,72.0,1,0,1,3,0,185.49,37.1,2
312,1,7.0,0,0,0,4,1,56.32,15.9,0
500,0,53.0,0,0,1,2,0,83.91,36.6,0
501,0,70.0,0,1,1,2,0,59.35,32.3,1
388,0,57.0,1,0,1,0,0,67.41,32.9,2
...,...,...,...,...,...,...,...,...,...,...
71,1,81.0,0,0,1,2,1,213.22,26.1,0
106,0,5.0,0,0,0,4,1,84.91,26.1,0
270,0,20.0,0,0,0,2,1,66.55,26.9,3
435,0,53.0,1,0,1,2,1,62.55,30.3,0


In [43]:
print(y_train)
print(y_train.value_counts())
# data for 1 is less then much when compare with 0

209    1
312    0
500    0
501    1
388    1
      ..
71     1
106    0
270    0
435    1
102    1
Name: stroke, Length: 497, dtype: int64
stroke
0    294
1    203
Name: count, dtype: int64


In [44]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state = 42)
model.fit(x_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [45]:
#evaluate the model
y_prediction = model.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.87      0.76      0.81        79
           1       0.66      0.80      0.73        46

    accuracy                           0.78       125
   macro avg       0.77      0.78      0.77       125
weighted avg       0.79      0.78      0.78       125



### The result is quite bad with the precision of disease is 66%, so i decide to move into
### another way about the value, i use smote to duplicate a haluminate data to balance

## Smote "Duplicate"

In [47]:
#download
!pip install imbalanced-learn

In [75]:
from imblearn.over_sampling import SMOTE

#Mix the data to ensure the distibution of value
data = data.sample(random_state = 42, frac = 1).reset_index(drop = True)

x_smote = data.drop(columns = 'stroke')
x_smote
y_smote = data['stroke']
y_smote

0       0
1       0
2       0
3       0
4       0
       ..
5105    0
5106    0
5107    0
5108    0
5109    0
Name: stroke, Length: 5110, dtype: int64

In [82]:
x_train_smote, x_test_smote, y_train_smote, y_test_smote = train_test_split(x_smote, y_smote,
                                                                            test_size = 0.2, 
                                                                            random_state = 42)
# only apply smote on the train data
smote = SMOTE(random_state = 42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train_smote, y_train_smote)
print(x_train_resampled)

      gender        age  hypertension  heart_disease  ever_married  work_type  \
0          0  29.000000             0              0             1          0   
1          1   4.000000             0              0             0          4   
2          1  45.000000             0              0             1          2   
3          1  69.000000             0              0             1          2   
4          0  50.000000             0              0             0          2   
...      ...        ...           ...            ...           ...        ...   
7775       0  80.000000             0              0             1          1   
7776       0  54.737326             1              0             1          2   
7777       1  48.252512             0              0             1          2   
7778       0  76.688980             0              0             1          2   
7779       1  59.198986             0              0             1          2   

      Residence_type  avg_g

In [77]:
print(y_train_resampled.value_counts(), y_train_resampled)

stroke
0    3890
1    3890
Name: count, dtype: int64 0       0
1       0
2       0
3       0
4       0
       ..
7775    1
7776    1
7777    1
7778    1
7779    1
Name: stroke, Length: 7780, dtype: int64


In [78]:
model_smote = RandomForestClassifier(random_state = 42)
model_smote.fit(x_train_resampled, y_train_resampled)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [83]:
# evaluate again
y_predict_smote = model_smote.predict(x_test)
print(classification_report(y_test, y_predict_smote))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90        79
           1       0.94      0.67      0.78        46

    accuracy                           0.86       125
   macro avg       0.89      0.82      0.84       125
weighted avg       0.87      0.86      0.86       125



In [84]:
import joblib

joblib.dump(model_smote, 'viegrand_stroke_model.pkl')


['viegrand_stroke_model.pkl']